In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import re
#from selenium.webdriver.chrome.service import Service

In [2]:
#path
#ser_obj='path'
#driver=webdriver.Chrome(service=ser_obj)

website='https://www.99acres.com/'
driver=webdriver.Chrome()
driver.get(website)
driver.maximize_window()
#driver.quit()

In [3]:
#List containing the city name for searching
city=['Bangalore (All)','Pune','Chennai (All)']

In [4]:
#declaring empty list for appending the respective info
building_name=[]
price=[]
region=[]
location=[]
building_type=[]
near_by=[]
availability=[]
#list declaration for second data frame
floor_area=[]
app_name=[]
app_price=[]
area_ty=[]
loc_det=[]
app_size=[]
comp_sta=[]
app_status=[]
app_region=[]

In [5]:
#This function is used to select enter the city name and go to the relevant page of the website
def selecting_city():
        #Selecting data for a particular city
    element=driver.find_element(by=By.XPATH, value='//input[@class="component__searchInput"]')
    #selecting the city from the list declared above
    element.send_keys(city[1])
    #setting a waiting time for the drop down to appear
    time.sleep(1.5)
    #selecting the first element from the dropdown
    driver.find_element(By.XPATH,value='//li[@id="0"]').click()
    #hitting search button
    driver.find_element(By.XPATH,value='//button[@type="submit" and @variant="primaryRegular" ]').click()

In [11]:
def scroll_page_():
    for i in range(15):
        driver.execute_script("window.scrollBy(0, 4200)")
        #setting sleep/waiting time for the page to load
        time.sleep(1)

In [7]:
#function for collecting the data
def collecting_data():
    #setting some default value if it does not extract  any value
    res_name=res_price=res_loc=avl=temp='unknown'
    
        #extracting web element containing all property details
    val=driver.find_elements(By.XPATH,value='//section[@data-hydration-on-demand="true"]')
    #extracting web element containing each property detail
    res=val[6].find_elements(By.XPATH,value='//div[@style="height: 82%;"]')
    #extracting property name
    res_name=res[1].find_elements(By.XPATH,value='//*[@class="projectTuple__projectName  projectTuple__pdWrap20 ellipsis"]')
    #extracting property price
    res_price=res[1].find_elements(By.XPATH,value='//span[@class="list_header_bold configurationCards__srpPriceHeading configurationCards__configurationCardsHeading"]')
    #extracting property location
    res_loc=res[1].find_elements(By.XPATH,value='//h2[@class="projectTuple__subHeadingWrap body_med ellipsis"]')
    #extracting property avilability
    avl=val[6].find_elements(By.XPATH,value='//div[@class="projectTuple__carouselTag projectTuple__carouselShortlistTagPostioning "]')
    #extracting property and thier options
    temp=val[6].find_elements(By.XPATH,value='//div[@class="configurationCards__cardsWrapper configurationCards__srpCardStyle"]')

    #loops to append elements to list
    for x,y,z in zip(res_name,res_price,res_loc):
        building_name.append(x.text)
        price.append(y.text)
        location.append(z.text)
    for i in range(len(avl)):
        availability.append(avl[i].text)
    for x in temp:
        building_type.append(x.text)

In [8]:
#alternate function to collect data
def collecting_data2():
        #sub section
    val_2=driver.find_elements(By.XPATH,value='//div[@class="srpTuple__cardWrap tupleCardWrap"]')
    #location details
    loc_detail=val_2[0].find_elements(By.XPATH,value='//td[@class="srpTuple__tdClassPremium"]')
    #name of property
    name2=val_2[0].find_elements(By.XPATH,value='//td[@class="srpTuple__propertyPremiumHeading srpTuple__spacer10 srpTuple__tdClasstwoPremium "]')
    #price and size details
    detail=val_2[0].find_elements(By.XPATH,value='//tr[@class="srpTuple__flexWrap"]')
    #avilability status
    avil=val_2[0].find_elements(By.XPATH,value='//div[@class="f10 Ng100 srpTuple__postedByText ellipsis"]')
    #area specification
    area_spec=val_2[0].find_elements(By.XPATH,value='//td[@class="srpTuple__col title_semiBold  "]')
    
    for x,y,z,a,b in zip(name2,detail,loc_detail,avil,area_spec):
        app_name.append(x.text)
        loc_det.append(z.text)
        app_price.append(str(y.text).split('\n')[0]) 
        app_size.append(str(y.text).split('\n')[-2])

        try:
            floor_area.append(str(y.text).split('\n')[2])
        except:
            floor_area.append('unknown')

        temp1=z.text.split()[-3:]
        temp1=' '.join(temp1)
        app_region.append(temp1)

        temp2=str(a.text).split()[3:-2]
        temp2=' '.join(temp2)
        temp2='Since '+str(temp2)
        comp_sta.append(temp2)
        app_status.append('Ready To Move')
        area_ty.append(b.text.split(')')[-1])

In [9]:
def next_page():
    time.sleep(1)
    #code for scrolling to page to a particular height for entering the next button
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight-4500);")
    time.sleep(1)
    #command for next button
    driver.find_elements(By.XPATH,value='//a[@class="list_header_bold" and contains(text(),"Next Page")]')[0].click()

In [12]:
#enter the page number
pages=12
for i in range(pages):
    if i==0:
        selecting_city()
    else:
        try:
            print('scrapping page =',i)
            scroll_page_()
            collecting_data()
            next_page()
        except:
            print('exception')
            scroll_page_()
            collecting_data2()
            next_page()

scrapping page = 1
scrapping page = 2
exception


IndexError: list index out of range

In [ ]:
#second dataframe
dat={'property_name':app_name,'size':app_size,'carpet_area':floor_area,'price':app_price,'avg_price':app_price,'location_details':loc_det,'region':app_region,'availability':app_status,'completion_status':comp_sta,'area_type':area_ty}
df1=pd.DataFrame(dat)

In [ ]:
df=df1

In [ ]:
#creating the dataset
data={'Property_name':building_name,'Price':price,'Location_details':location,'Building_Type':building_type}
df=pd.DataFrame(data)
#creating the feature called region (extracted the last 2 words from the loaction details column)
for i in range(len(df)):
    region.append(df['Location_details'][i].split()[-2]+' '+df['Location_details'][i].split()[-1])
df['region']=region
#dropping the row with null in building type
df.drop(df[df['Building_Type']==''].index,axis=0,inplace=True)
df['availability_of_building']=availability
#extracting current building status and info regarding the finishing/finished date of construction
df['availability']=df['availability_of_building'].map(lambda x:x.split('\n')[0])
df['completion_status']=df['availability_of_building'].map(lambda x:x.split('\n')[-1])
df.drop('availability_of_building',axis=1,inplace=True)
df['completion_status']=df['completion_status'].str.replace('Ready To Move','unknown')

#resetting the index
df=df.reset_index()
df.drop('index',axis=1,inplace=True)

In [ ]:
df

In [ ]:
size1=[]
size2=[]
size3=[]

In [ ]:
for i in range(len(df)):
    text=str(df['Building_Type'][i]).lower()
    ob1=re.compile(r'\d{1}\sbhk*|land')
    k=ob1.findall(text)
    if len(k)==0:
        size1.append('nil')
        size2.append('nil')
        size3.append('nil')
    elif len(k)==1:
        size1.append(ob1.findall(text)[0]+" "+ob1.split(text)[1])
        size2.append('nil')
        size3.append('nil')
    elif len(k)==2:
        size1.append(ob1.findall(text)[0]+" "+ob1.split(text)[1])
        size2.append(ob1.findall(text)[1]+" "+ob1.split(text)[2])
        size3.append('nil')
    else:
        size1.append(ob1.findall(text)[0]+" "+ob1.split(text)[1])
        size2.append(ob1.findall(text)[1]+" "+ob1.split(text)[2])
        size3.append(ob1.findall(text)[2]+" "+ob1.split(text)[3])

In [ ]:
#adding the size details to the data frame
df['size1']=size1
df['size2']=size2
df['size3']=size3

In [ ]:
name_property=[]
size=[]
carpet_area=[]
price_range=[]
avg_price=[]
location_details=[]
region_1=[]
availability_stat=[]
completion_stat=[]
area_type=[]

In [ ]:
for i in range(len(df)):
    if df['size1'][i]!='nil':
        try:
            t1=df['Property_name'][i]                 #gets name of the property
            t2=df['size1'][i].split('\n')[0]          #get size details
            t3=df['size1'][i].split('\n')[3]          #gets carpet area
            t4=df['size1'][i].split('\n')[5]          #gets price range
            t5=df['Price'][i]                         #gets average price
            t6=df['Location_details'][i]              #gets location details
            t7=df['region'][i]                        #gets the region
            t8=df['availability'][i]                  #gets the availability status
            t9=df['completion_status'][i]             #gets completion status
            t10=df['size1'][i].split('\n')[2]         #gets area type info
        except:
            t1=t2=t3=t4=t5=t6=t7=t8=t9=t10='unknown'
        name_property.append(t1)
        size.append(t2)
        carpet_area.append(t3)
        price_range.append(t4)
        avg_price.append(t5)
        location_details.append(t6)
        region_1.append(t7)
        availability_stat.append(t8)
        completion_stat.append(t9)
        area_type.append(t10)
        
    if df['size2'][i]!='nil':
        try:
            t1=df['Property_name'][i]                 #gets name of the property
            t2=df['size2'][i].split('\n')[0]          #get size details
            t3=df['size2'][i].split('\n')[3]          #gets carpet area
            t4=df['size2'][i].split('\n')[5]          #gets price range
            t5=df['Price'][i]                         #gets average price
            t6=df['Location_details'][i]              #gets location details
            t7=df['region'][i]                        #gets the region
            t8=df['availability'][i]                  #gets the availability status
            t9=df['completion_status'][i]             #gets completion status
            t10=df['size2'][i].split('\n')[2]         #gets area type info
        except:
            t1=t2=t3=t4=t5=t6=t7=t8=t9=t10='unknown'
        name_property.append(t1)
        size.append(t2)
        carpet_area.append(t3)
        price_range.append(t4)
        avg_price.append(t5)
        location_details.append(t6)
        region_1.append(t7)
        availability_stat.append(t8)
        completion_stat.append(t9)
        area_type.append(t10)
        
    if df['size3'][i]!='nil':
        try:
            t1=df['Property_name'][i]                 #gets name of the property
            t2=df['size3'][i].split('\n')[0]          #get size details
            t3=df['size3'][i].split('\n')[3]          #gets carpet area
            t4=df['size3'][i].split('\n')[5]          #gets price range
            t5=df['Price'][i]                         #gets average price
            t6=df['Location_details'][i]              #gets location details
            t7=df['region'][i]                        #gets the region
            t8=df['availability'][i]                  #gets the availability status
            t9=df['completion_status'][i]             #gets completion status
            t10=df['size3'][i].split('\n')[2]         #gets area type info
        except:
            t1=t2=t3=t4=t5=t6=t7=t8=t9=t10='unknown'
        name_property.append(t1)
        size.append(t2)
        carpet_area.append(t3)
        price_range.append(t4)
        avg_price.append(t5)
        location_details.append(t6)
        region_1.append(t7)
        availability_stat.append(t8)
        completion_stat.append(t9)
        area_type.append(t10)

In [ ]:
#creating the final data set
final_data=pd.DataFrame()
final_data['property_name']=name_property
final_data['size']=size
final_data['carpet_area']=carpet_area
final_data['price']=price_range
final_data['avg_price']=avg_price
final_data['location_details']=location_details
final_data['region']=region_1
final_data['availability']=availability_stat
final_data['completion_status']=completion_stat
final_data['area_type']=area_type

In [ ]:
#dropping the data's which had issues during the transformation
final_data.drop(final_data[final_data['property_name']=='unknown'].index,axis=0,inplace=True)
final_data=final_data.reset_index()
final_data.drop('index',axis=1,inplace=True)

In [ ]:
final_data

In [ ]:
final_data.to_csv('dataframe1.csv')

In [ ]:
data=final_data.copy()
data2=df1.copy()

In [ ]:
data2.to_csv('dataframe2.csv')

In [ ]:
final=pd.concat([data,data2],axis=0).reset_index()

In [ ]:
final.drop('index',axis=1,inplace=True)

In [ ]:
final

In [ ]:
final.to_csv('99acres_new.csv')
df3=pd.read_csv('99acres.csv')
data_final=final.copy()

In [ ]:
data_final.drop(data_final[data_final['property_name']==''].index,axis=0,inplace=True)
data_final.drop_duplicates(inplace=True)
data_final=data_final.reset_index()
data_final.drop('index',axis=1,inplace=True)

In [ ]:
data_final

In [ ]:
data_final.to_csv('clean_data_Uttarpradesh.csv')
data=pd.read_csv('clean_data_Uttarpradesh.csv')

In [ ]:
#dropping duplicates 
data.drop_duplicates(inplace=True)
#dropping all the data where price feature is having price on request string
data.drop((data[data['avg_price']=='Price on request'].index),axis=0,inplace=True)    #only less than 20 records so we delete them
#now in the price column also we are having the price on request we will now replace it with the avg price 

In [ ]:
#splitting it into 2 df
dd1=data[data['price']=='price on request'] 
dd2=data[data['price']!='price on request']
#replacing the price column with value in the avg price columns
dd1['price']=dd1['avg_price']

In [ ]:
clean1=pd.concat([dd1,dd2],axis=0).reset_index(drop=bool)
#dropping the the avg_price column
clean1.drop(['Unnamed: 0','avg_price'],axis=1,inplace=True)

In [ ]:
#splitting the range of price into two features low and high
clean1['price_low']=clean1['price'].map(lambda x:x.split('-')[0])
clean1['price_high']=clean1['price'].map(lambda x:x.split('-')[-1])
#splitting the carpet area into 2 feature to find the avg
clean1['area_low']=clean1['carpet_area'].map(lambda x:x.split('-')[0])
clean1['area_high']=clean1['carpet_area'].map(lambda x:x.split('-')[-1])

In [ ]:
#removing the unit of square ft
clean1['area_low']=clean1['area_low'].map(lambda x:x.split()[0])
clean1['area_high']=clean1['area_high'].map(lambda x:x.split()[0])

In [ ]:
#we have to remove the comma to transform into a numeric value
clean1['area_low']=clean1['area_low'].map(lambda x:x.replace(',',''))
clean1['area_high']=clean1['area_high'].map(lambda x:x.replace(',',''))
#dropping the record which is having carpet area =unknown
clean1.drop(clean1[clean1['carpet_area']=='unknown'].index,axis=0,inplace=True)
#converting the area into numerical value
clean1['area_high']=pd.to_numeric(clean1['area_high'])
clean1['area_low']=pd.to_numeric(clean1['area_low'])

In [ ]:
#creating a new feature avg_area based on the avg of low and high area
clean1['avg_area']=(clean1['area_low']+clean1['area_high'])/2

In [ ]:
clean1.drop(['area_low','carpet_area','area_high'],axis=1,inplace=True)
clean1.reset_index(drop=bool,inplace=True)

In [ ]:
#to convert into lower case
clean1['price_high']=clean1['price_high'].str.lower()
clean1['price_low']=clean1['price_low'].str.lower()
clean1['price_low']=clean1['price_low'].map(lambda x:' '.join(x.split()[1:]))

In [ ]:
def price_to_number(x):
    condition=re.compile(r'[+-]?([0-9]*[.])?[0-9]+')
    out=condition.search(x)
    k=float(out.group())
    return k

In [ ]:
def size_extractor(x):
    condition=re.compile(r'[+-]?([0-9]*[,])?[0-9]+')
    out=condition.search(x)
    k=float(out.group())
    return k

In [ ]:
#defining 2 feature dummy
clean1['p_low']=clean1['p_high']=0

In [ ]:
#to find out the prices are in crores or in lakhs
for i in range(len(clean1)):
    if clean1['price_low'][i].split()[-1]!='l' and clean1['price_high'][i].split()[-1]=='cr':
        #print('both r crore')
        clean1['p_low'][i]=100
        clean1['p_high'][i]=100
    elif clean1['price_low'][i].split()[-1]=='l' and clean1['price_high'][i].split()[-1]=='cr':
        #print('low is lakh high is crore')
        clean1['p_low'][i]=1
        clean1['p_high'][i]=100
    else:
        #print('both r lack')
        clean1['p_low'][i]=1
        clean1['p_high'][i]=1

In [ ]:
clean1['price_new_high']=clean1['price_high'].map(price_to_number)
clean1['price_new_low']=clean1['price_low'].map(price_to_number)

In [ ]:
#multiplying elements to convert price in bring it in lakhs
clean1['price_new_high']=clean1['price_new_high']*clean1['p_high']
clean1['price_new_low']=clean1['price_new_low']*clean1['p_low']

In [ ]:
#average price
clean1['avg_price']=(clean1['price_new_high']+clean1['price_new_low'])/2
clean1.drop(['price','price_low','price_high','p_low','p_high','price_new_high','price_new_low'],axis=1,inplace=True)

In [ ]:
clean1['size']=clean1['size'].str.lower()
clean1['region']=clean1['region'].str.lower()

In [ ]:
#replacing all records (size column) which doesnt have the key word bhk in it with open plot
sub='bhk'
for i in range(len(clean1)):
    if sub not in clean1['size'][i]:
        clean1['size'][i]='open plot'
    else:
        continue

In [ ]:
clean1['price_per_sq_ft']=clean1['avg_price']/clean1['avg_area']

In [ ]:
#checking for duplicates
clean1.duplicated().sum()
clean1.drop_duplicates(inplace=True)
clean1.reset_index(drop=bool,inplace=True)

In [ ]:
clean_data=clean1
clean_data.to_csv('clean_data.csv')

In [ ]:
#next

In [ ]:
#to extract the year from the completion status feature
def search(x):
    ob1=re.compile(r'\d{4}')
    ob2=ob1.findall(x)
    if(len(ob2)!=0):
        temp=ob2[0]
    else:
        temp=2023
    return temp

In [ ]:
#to extract the year from the completion status feature
def rooms(x):
    ob1=re.compile(r'\d')
    ob2=ob1.findall(x)
    if(len(ob2)!=0):
        temp=ob2[0]
    else:
        temp=0
    return temp

In [ ]:
def extract_age(x):
    temp=2023-int(x)
    return temp

In [ ]:
data['completion_status']=data['completion_status'].map(search)
data['age_of_building']=data['completion_status'].map(extract_age)

In [ ]:
data=data.drop_duplicates(None).reset_index()
data['no_rooms']=data['size'].map(rooms)
data.drop(['completion_status','index','size'],axis=1,inplace=True)

In [ ]:
data['price_per_sq']=data['price_per_sq_ft']
data.drop('price_per_sq_ft',axis=1,inplace=True)